In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('nytaxi2022.csv')
df.head()

/var/folders/7p/pgzfcdb953s5cgb46bb6dv6h0000gn/T/ipykernel_45879/1526527318.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('nytaxi2022.csv')


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,01/01/2022 12:35:40 AM,01/01/2022 12:53:29 AM,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,01/01/2022 12:33:43 AM,01/01/2022 12:42:07 AM,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,01/01/2022 12:53:21 AM,01/01/2022 01:02:19 AM,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,01/01/2022 12:25:21 AM,01/01/2022 12:35:23 AM,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,01/01/2022 12:36:48 AM,01/01/2022 01:14:20 AM,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [3]:
df.shape

(39656098, 19)

In [4]:
import pandas as pd

# convert "" or whitespace-only cells to NA so pandas counts them as missing
df = df.replace(r"^\s*$", pd.NA, regex=True)

The code above just turns empty or whitespace-only strings into proper missing values. 

As CSV files often store "empty" cells as "" or spaces. By default, pandas won't treat " " as missing. Converting them to pd.NA makes your missingness checks and imputations work correctly. 

# Analyzing and cleaning the data

In [5]:
# number of missing row within a columns
df.isna().sum()            # per-column counts

VendorID                       0
tpep_pickup_datetime           0
tpep_dropoff_datetime          0
passenger_count          1368303
trip_distance                  0
RatecodeID               1368303
store_and_fwd_flag       1368303
PULocationID                   0
DOLocationID                   0
payment_type                   0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge     1368303
airport_fee              1368303
dtype: int64

In [6]:
N = len(df)
cols = ["passenger_count","RatecodeID","store_and_fwd_flag","congestion_surcharge","airport_fee"]

miss_tbl = (
    df[cols].isna().sum()
      .to_frame("missing")
      .assign(pct=lambda t: (t["missing"]/N*100).round(2))
)
miss_tbl  # quick table

# Are the *same* rows missing all five?
co_missing = df[cols].isna().all(axis=1)
co_missing.sum(), (co_missing.mean()*100).round(2)  # count & %

(np.int64(1368303), np.float64(3.45))

The code computes how much data is missing in a specific set of columns, then checks if it is the same rows missing across all of them. Firstly, it builds `miss_tbl`: for each column in ["passenger_count", "RatecodeID", "store_and_fwd_flag", "congestion_surcharge", "airport_fee"], it counts missing values and adds a percentage of total rows (N). Next, co_missing = df[cols].isna().all(axis=1) flags rows where all five of those columns are missing; co_missing.sum() gives the number of such rows and `co_missing.mean()*100` gives their percentage. 
This quickly shows both per-column gaps and whether there's a shared "co-missing" pattern-useful for deciding whether to impute together, drop those rows, or treat them specifically.

Output confirms that the same 1,368,303 (~3.45%) are missing values in these columns: passenger_count, RatecodeID, store_and_fwd_flag, congestion_surcharge, airport_fee. Since co_missing.sum() is 1,368,303 and that matches the per-column missing counts you showed earlier, it means the same 1,368,303 rows are missing values in every one of those columns (≈3.45% of the dataset). There are no rows where only a subset of those columns is missing.

In [7]:
TARGET = "total_amount"

allowed = ["tpep_pickup_datetime","tpep_dropoff_datetime",
    "passenger_count","trip_distance","RatecodeID",
    "PULocationID","DOLocationID","payment_type","extra"]

leakage = ["fare_amount","mta_tax","tip_amount","tolls_amount",
    "improvement_surcharge","congestion_surcharge","airport_fee", TARGET]

present = [c for c in df.columns if c in allowed]
leak_present = [c for c in df.columns if c in leakage and c != TARGET]

# optional: remove near-constant and very-missing among the allowed set
missing = df[present].isna().mean().sort_values(ascending=False)
const_like = [c for c in present if df[c].nunique(dropna=True) <= 1]
very_missing = [c for c in present if missing[c] > 0.40] #40% threshold; tweak

keep = [c for c in present if c not in const_like and c not in very_missing]

print("Keep: ", keep)
print("DROP (leakage): ", leak_present)
print("DROP (constant): ", const_like)
print("DROP (very missing): ", very_missing)

Keep:  ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'PULocationID', 'DOLocationID', 'payment_type', 'extra']
DROP (leakage):  ['fare_amount', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'airport_fee']
DROP (constant):  []
DROP (very missing):  []


Builds a *clean feature list* and tells you what to keep vs drop, with guards against target leakage and junk columns. 

* TARGET = your label;
- allowed = features you are allowed to use;
- leakage = columns that directly compost `total_amount` (and must not be features)

* present = intersection of allowed with the columns actually in df.
* leak_present = any leakage columns that exist in df (to confirm whether the column is excluded)

It then scores the present features:

missing = percent missing per column,

const_like = columns with ≤1 distinct non-NA value (no signal),

very_missing = features with >40% missing (threshold you can tweak).

keep = present minus (const_like ∪ very_missing). That’s your final whitelist.

## Engineered feature plus stabilisation before test split

In [8]:
import numpy as np
m = df[keep + [TARGET]].copy()
for c in ["tpep_pickup_datetime","tpep_dropoff_datetime"]:
    m[c] = pd.to_datetime(m[c], errors="coerce", utc=True)

m["trip_duration_min"] = (m["tpep_dropoff_datetime"] - m["tpep_pickup_datetime"]).dt.total_seconds()/60
m["pickup_hour"] = m["tpep_pickup_datetime"].dt.hour.astype("Int8")
m["pickup_dow"]  = m["tpep_pickup_datetime"].dt.dayofweek.astype("Int8")
m["pickup_month"] = m["tpep_pickup_datetime"].dt.month.astype("Int8")   # <--- added

# --- add stabilization step here ---
m = m[(m["total_amount"] > 0) &
     (m["total_amount"] < 300) &
     (m["trip_distance"] > 0) &
     (m["trip_duration_min"] > 0) &
     (m["trip_duration_min"] <= 180)]


/var/folders/7p/pgzfcdb953s5cgb46bb6dv6h0000gn/T/ipykernel_45879/2672286999.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  m[c] = pd.to_datetime(m[c], errors="coerce", utc=True)
/var/folders/7p/pgzfcdb953s5cgb46bb6dv6h0000gn/T/ipykernel_45879/2672286999.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  m[c] = pd.to_datetime(m[c], errors="coerce", utc=True)


NOTE: The stabilization step removes implausible fares (e.g. >$300), zero/negative distances,
and unrealistic trip durations. Without it, extreme outliers (~$400k fares) 
would dominate RMSE and make the model look much worse than it actually is.

In [9]:
print("df: ", df.shape)

df:  (39656098, 19)


In [10]:
print("m. ", m.shape)

m.  (38795244, 14)


## Sanity + fix dtype

In [11]:
# Make sure numerics are truly numeric (bad strings -> NaN)
for c in ["passenger_count","trip_distance","extra","total_amount"]:
    m[c] = pd.to_numeric(m[c], errors="coerce")

# Keep engineered ints as floats so sklearn sees NaN correctly
m["trip_duration_min"] = m["trip_duration_min"].astype("float32")
m["pickup_hour"]       = m["pickup_hour"].astype("float32")
m["pickup_dow"]        = m["pickup_dow"].astype("float32")

# add pickup_month (1–12)
m["pickup_month"]      = m["tpep_pickup_datetime"].dt.month.astype("float32")

# Cast categoricals
for c in ["RatecodeID","PULocationID","DOLocationID","payment_type"]:
    m[c] = m[c].astype("category")

## Random train test split

In [14]:
from sklearn.model_selection import train_test_split

NUM = [
    "passenger_count",
    "trip_distance",
    "extra",
    "trip_duration_min",
    "pickup_hour",
    "pickup_dow",
    "pickup_month"
]
CAT = ["RatecodeID", "PULocationID", "DOLocationID", "payment_type"]
TARGET = "total_amount"

X, y = m[NUM + CAT], m[TARGET].astype("float32")
print("final X,y:", X.shape, y.shape)

Xtr, Xte, ytr, yte = train_test_split(
    X, y, test_size=0.30, random_state=42
)

# dropna for safety
Xtr_clean = Xtr.dropna(); ytr_clean = ytr.loc[Xtr_clean.index]
Xte_clean = Xte.dropna(); yte_clean = yte.loc[Xte_clean.index]

final X,y: (38795244, 11) (38795244,)


In [15]:
len(Xtr), len(Xte), len(ytr), len(yte)

(27156670, 11638574, 27156670, 11638574)

## Sanity-check

In [16]:
yte.describe(percentiles=[.95,.99,.999])
(yte > 300).mean()   # share of crazy-high fares

np.float64(0.0)

 ### 📝 Why I Changed the Encoding Strategy  

Initially, I one-hot encoded **all categorical features**, including `PULocationID` and `DOLocationID`.  
These columns have **hundreds of unique values**, so one-hot encoding created **tens of thousands of dummy columns**.  

This caused two major issues:  
- 🚨 **Feature explosion** → the transformed dataset became extremely wide, making Random Forest training unbearably slow (hours).  
- 🚨 **Memory overhead** → every tree split had to scan across all those sparse dummy features, wasting CPU and RAM.  

---

### ✅ Solution  

- Keep **one-hot encoding (OHE)** only for **low-cardinality categoricals** (`RatecodeID`, `payment_type`).  
- Pass through **high-cardinality features** (`PULocationID`, `DOLocationID`) as numeric integers instead of OHE.  

---

### 💡 Why this works  

- 🌳 Tree-based models (Random Forest, LightGBM) handle numeric IDs directly and can split on them like categorical values.  
- 📉 The feature space stays compact (a few dozen features instead of tens of thousands).  
- ⚡ Training time drops from **hours → minutes** with no meaningful loss in predictive power.  

## Baseline Ridge Regression

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Preprocessor (Ridge needs scaling + OHE)
pre_ridge = ColumnTransformer([
    ("num", Pipeline([
        ("sc", MaxAbsScaler())
    ]), NUM),
    ("cat_low", OneHotEncoder(handle_unknown="ignore", min_frequency=5), ["RatecodeID", "payment_type"]),
    ("cat_high", "passthrough", ["PULocationID", "DOLocationID"]),
], sparse_threshold=1.0)

ridge = Pipeline([
    ("pre", pre_ridge),
    ("est", Ridge(alpha=1.0, solver="sag", max_iter=2000, random_state=42))
])

ridge.fit(Xtr_clean, ytr_clean)
pred = ridge.predict(Xte_clean)

mae  = mean_absolute_error(yte_clean, pred)
mse  = mean_squared_error(yte_clean, pred)
rmse = np.sqrt(mse)
r2   = r2_score(yte_clean, pred)

print(f"[Ridge] MAE={mae:.3f}  RMSE={rmse:.3f}  R2={r2:.4f}")

[Ridge] MAE=3.779  RMSE=7.263  R2=0.8094


## Baseline Random Forest

In [18]:
# Sanity check - Random forest 

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

# --- Minimal preprocessor (same structure as baseline) ---
pre_rf = ColumnTransformer([
    ("num", "passthrough", NUM),
    ("cat_low", OneHotEncoder(handle_unknown="ignore", min_frequency=5), ["RatecodeID", "payment_type"]),
    ("cat_high", "passthrough", ["PULocationID", "DOLocationID"]),
])

# --- Tiny RF config for sanity check ---
rf_debug = Pipeline([
    ("pre", pre_rf),
    ("est", RandomForestRegressor(
        n_estimators=5,      # just 5 trees
        max_depth=6,         # shallow trees
        n_jobs=-1,
        random_state=42,
        verbose=1
    ))
])

# --- Small sample (e.g. 20k rows) ---
Xtr_debug = Xtr_clean.sample(n=20_000, random_state=42)
ytr_debug = ytr_clean.loc[Xtr_debug.index]

rf_debug.fit(Xtr_debug, ytr_debug)
pred_debug = rf_debug.predict(Xte_clean.head(5))  # predict just 5 rows

print("Sample predictions:", pred_debug)
print("Sanity check complete ✅")

Sample predictions: [35.46774237 73.04246339 73.04246339 15.94616308 14.00170702]
Sanity check complete ✅


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s finished


In [19]:
from sklearn.ensemble import RandomForestRegressor

# Preprocessor (OHE only low-cardinality)
pre_rf = ColumnTransformer([
    ("num", "passthrough", NUM),
    ("cat_low", OneHotEncoder(handle_unknown="ignore", min_frequency=20), ["RatecodeID", "payment_type"]),
    ("cat_high", "passthrough", ["PULocationID", "DOLocationID"]),  # <-- key change
])

rf = Pipeline([
    ("pre", pre_rf),
    ("est", RandomForestRegressor(
        n_estimators=50,
        max_depth=12,
        min_samples_leaf=20,
        max_features=0.5,
        n_jobs=-1,
        random_state=42,
        verbose=1
    ))
])

Xtr_small = Xtr_clean.sample(n=200_000, random_state=42)  # smaller subsample for speed
ytr_small = ytr_clean.loc[Xtr_small.index]

rf.fit(Xtr_small, ytr_small)
pred = rf.predict(Xte_clean)

mae  = mean_absolute_error(yte_clean, pred)
mse  = mean_squared_error(yte_clean, pred)
rmse = np.sqrt(mse)
r2   = r2_score(yte_clean, pred)

print(f"[RandomForest] MAE={mae:.3f}  RMSE={rmse:.3f}  R2={r2:.4f}")

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    1.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    1.9s


[RandomForest] MAE=1.461  RMSE=3.518  R2=0.9553


[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:    3.1s finished


## Baseline Lightgbm Model

In [23]:
from lightgbm import LGBMRegressor

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# --- Preprocessor ---
pre_lgbm = ColumnTransformer([
    ("num", "passthrough", NUM),
    ("cat_low", OneHotEncoder(handle_unknown="ignore", min_frequency=10), ["RatecodeID", "payment_type"]),
    ("cat_high", "passthrough", ["PULocationID", "DOLocationID"]),
], sparse_threshold=1.0)

# --- Pipeline ---
lgbm = Pipeline([
    ("pre", pre_lgbm),
    ("est", LGBMRegressor(
        n_estimators=300,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        n_jobs=-1,
        random_state=42
    ))
])

# --- Train on subsample for speed ---
Xtr_small = Xtr_clean.sample(n=200_000, random_state=42)
ytr_small = ytr_clean.loc[Xtr_small.index]

lgbm.fit(Xtr_small, ytr_small)

pred = lgbm.predict(Xte_clean)

mae  = mean_absolute_error(yte_clean, pred)
mse  = mean_squared_error(yte_clean, pred)
rmse = np.sqrt(mse)
r2   = r2_score(yte_clean, pred)

print(f"[LightGBM] MAE={mae:.3f}  RMSE={rmse:.3f}  R2={r2:.4f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002943 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 956
[LightGBM] [Info] Number of data points in the train set: 200000, number of used features: 19
[LightGBM] [Info] Start training from score 21.468120


/Users/chris/Library/CloudStorage/Dropbox/work/NUS/DSA5208/Assignment_one/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] MAE=1.354  RMSE=3.311  R2=0.9604


### 📊 Baseline Model Comparison  

| Model        | MAE (↓) | RMSE (↓) | R² (↑) | Notes |
|--------------|---------|----------|--------|-------|
| **Ridge**    | 3.779   | 7.263    | 0.809  | Linear baseline, decent sanity check but underfits nonlinear patterns. |
| **RandomForest** | 1.461   | 3.518    | 0.955  | Strong performance, captures nonlinearities, but slower to train. |
| **LightGBM** | 1.354   | 3.311    | 0.960  | Best overall: fast, accurate, and efficient with large datasets. |

---

### 🔎 Key Takeaways
- Ridge provides a **baseline** but struggles with complex relationships.  
- Random Forest captures more detail, but can be **computationally heavy**.  
- LightGBM edges out Random Forest with **better accuracy and faster training**, making it the most practical baseline for large-scale experiments.  